In [2]:
import sys
sys.path.append('/home/gangda/workspace/gnn-parallel-inference')
from utils import elapse, adj_norm

import argparse
import os
import time

import torch
from torch_sparse import matmul
from tqdm import tqdm
import torch.nn.functional as F

from torch_geometric.data import ClusterData, ClusterLoader, NeighborSampler
from torch_geometric.nn import SAGEConv, GCNConv, Linear
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(GCN, self).__init__()

        self.num_layers = num_layers

        self.lins = torch.nn.ModuleList()
        self.lins.append(Linear(in_channels, hidden_channels, weight_initializer='glorot'))
        for _ in range(self.num_layers - 2):
            self.lins.append(Linear(hidden_channels, hidden_channels, weight_initializer='glorot'))
        self.lins.append(Linear(hidden_channels, out_channels, weight_initializer='glorot'))

    @torch.no_grad()
    def forward(self, x, adj_t, is_normed=False):
        if not is_normed:
            adj_t = adj_norm(adj_t)
        for i, lin in enumerate(self.lins):
            x = lin(x)
            x = matmul(adj_t, x)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return torch.log_softmax(x, dim=-1)

In [20]:
from torch_geometric.datasets import Reddit2

start_time = time.perf_counter()

path = os.environ.get('DATA_DIR')

"""pyg datasets"""
# path = os.path.join(path, 'pyg', 'Reddit2')
# dataset = Reddit2(path, transform=T.ToSparseTensor())

"""ogb datasets"""
path = os.path.join(path, 'ogb')
# dataset = PygNodePropPredDataset(name='ogbn-papers100M', root=path)
dataset = PygNodePropPredDataset(name='ogbn-products', root=path, transform=T.ToSparseTensor())

torch.manual_seed(12345)
data = dataset[0]
adj_t = adj_norm(data.adj_t)

In [ ]:
data.x

In [17]:
device = 'cuda'
start = time.perf_counter()
x, adj_t, y = data.x.to(device), adj_t.to(device), data.y.to(device)
model = GCN(data.x.size(-1), 128, dataset.num_classes, num_layers=2).to(device)

if device != 'cpu':
    end = time.perf_counter()
    print(f'Data Transfer Time (CPU->GPU):',  elapse(start, end))

for epoch in range(1, 1001):
    model.eval()

    start = time.perf_counter()

    # optimizer.zero_grad()
    out = model(x, adj_t, is_normed=True)
    # loss = F.nll_loss(out, y.squeeze(1))
    # loss.backward()
    # optimizer.step()

    end = time.perf_counter()
    print(f'Epoch: {epoch:03d}, Time:{elapse(start, end)}')

Data Transfer Time (CPU->GPU): 103 ms
Epoch: 001, Time:0 ms
Epoch: 002, Time:0 ms
Epoch: 003, Time:0 ms
Epoch: 004, Time:0 ms
Epoch: 005, Time:0 ms
Epoch: 006, Time:0 ms
Epoch: 007, Time:0 ms
Epoch: 008, Time:0 ms
Epoch: 009, Time:0 ms
Epoch: 010, Time:0 ms
Epoch: 011, Time:0 ms
Epoch: 012, Time:0 ms
Epoch: 013, Time:0 ms
Epoch: 014, Time:0 ms
Epoch: 015, Time:0 ms
Epoch: 016, Time:0 ms
Epoch: 017, Time:0 ms
Epoch: 018, Time:0 ms
Epoch: 019, Time:0 ms
Epoch: 020, Time:0 ms
Epoch: 021, Time:0 ms
Epoch: 022, Time:0 ms
Epoch: 023, Time:0 ms
Epoch: 024, Time:0 ms
Epoch: 025, Time:0 ms
Epoch: 026, Time:0 ms
Epoch: 027, Time:0 ms
Epoch: 028, Time:0 ms
Epoch: 029, Time:0 ms
Epoch: 030, Time:0 ms
Epoch: 031, Time:0 ms
Epoch: 032, Time:0 ms
Epoch: 033, Time:0 ms
Epoch: 034, Time:0 ms
Epoch: 035, Time:0 ms
Epoch: 036, Time:0 ms
Epoch: 037, Time:0 ms
Epoch: 038, Time:0 ms
Epoch: 039, Time:0 ms
Epoch: 040, Time:0 ms
Epoch: 041, Time:0 ms
Epoch: 042, Time:0 ms
Epoch: 043, Time:0 ms
Epoch: 044, Time

KeyboardInterrupt: 